# BELTRANS Data Integration Analysis

This notebook contains an analysis of data from the BELTRANS project.

Before this analysis, each data source was loaded into different "namespaces" of our **Blazegraph** triple store. Data sources which are already in RDF were directly uploaded to the triple store, heterogeneous data were first mapped via rml.io.

We use `SPARQL` queries to investigate for the different data sources how they overlap. The goal is to identify optimal integration strategies. 

In [5]:
import sys
sys.path.insert(0, './utils/')

from SPARQLWrapper import SPARQLWrapper, TURTLE, JSON
import sparql_utils

endpointWikidata = SPARQLWrapper('http://wikibase-test-srv01.kbr.be/sparql/namespace/wikidata-belgians/sparql')
endpointBnf = SPARQLWrapper('http://wikibase-test-srv01.kbr.be/sparql/namespace/bnf-authors-persons/sparql')
endpointNTA = SPARQLWrapper('http://wikibase-test-srv01.kbr.be/sparql/namespace/kb-authors-nta/sparql')

## Goals
We specify the goals of data integration so we can measure and validate the outcome of our integration activities.

1. We need to get Belgian authors we do not have already in our KBR authors dataset 
  * *-> This increases our corpus*
2. We need to enrich KBR authors without nationality with any found nationality 
  * *-> This marks authors in our dataset as non-Belgians and for the future we can prioritize them lower*
3. We need to enrich KBR authors without nationality with Belgian nationality if they have it 
  * *-> This marks authors in our dataset as Belgians and for the future we can prioritize them*
4. We need to enrich KBR authors with missing ISNI and VIAF numbers
  * *-> This increaes interoperability (similar for the following two goals)*
5. We need to enrich KBR authors with identifiers from other libraries
6. We need to enrich KBR authors with common identifiers such as Wikidata or DBpedia


## Approach
We define how we want to achieve the goals using existing data sources.

Each data source is represented as RDF and can be queries using SPARQL. If a data source is not yet represented in RDF we lift it to RDF using the framework [rml.io](https://rml.io).

Generally, we have two phases. Firstly we identify missing information based on linking, and secondly we query information to enriche our KBR data. The second phase may involve manual tasks.

Please note that ISNI and VIAF can be used in two ways: either as identifiers within KBR data and other data sources which enable linking, or as dedicated data source as ISNI (issued by OCLC) and VIAF provide data dumps themselves.

### Complement KBR data with new Belgians
To achieve goal *1.* we can ask the following question per data source to identify relevant data:

* Which authors have Belgian nationality?

Therefore we identify Belgians in the source. However, some of these identififed Belgians may already be in our KBR data. Using identifiers such as ISNI or VIAF which are present in the KBR data we can narrow the initial search result down to a smaller dataset. This smaller dataset can then be further analyzed automatically regarding overlap to KBR data based on entity linking techniques, manual checks or a combination of both.


### Identify unknown nationalities in KBR data
To achieve goal *2.* and *3.* we can ask the following questions per data source to identify relevant data.

* Which KBR authors without nationality are actually Belgians?
* Which KBR authors with other than a Belgian nationality are *also* Belgians?
* Which KBR authors without nationality do have a known nationality other than Belgian?

KBR authors with unknown nationality can be supplemented with nationality information based on linking KBR authors to data sources based on library-related identifiers such as ISNI and VIAF or common identifiers such as Wikidata or DBpedia. 

### Enrich KBR data with authority-related identifiers
To achieve goal *4.* we can ask the following questions per data source to identify relevant data.

* Which (Belgian) KBR authors are present in a data source and have a VIAF or ISNI number?

Since obvously we only use KBR authors without such identifiers as starting point, we have to link to data sources via other identifiers such as library identifiers or Wikidata to find matches and extract possible VIAF and ISNI numbers.

### Enrich KBR data with identifiers from other libraries
To achieve goal *5.* ..

### Enrich KBR data with common identifiers



## Wikidata Belgians

This is a wikidata dump of Belgians optionally with BnF, KB NTA, DNB and ISNI identifiers which was manually enriched with KBR identifiers.


**Phase 1: identification**


All wikidata Belgians who are NOT in BnF =>
All wikidata Belgians who 

Overlap between BnF Belgians and wikidata Belgians could enrich BnF Belgians with information from Wikidata. However, so far we do not have the need or concrete properties we have to complete.

Which BnF authors without nationality are actually Belgians?
Which BnF authors with other than a Belgian nationality are *also* Belgian?

All BnF Belgians who are - based on BnF ID - not in the wikidata Belgians dump -> enrich wikidata Belgians with BnF ID
All BnF Belgians who are - based on ISNI - not in the wikidata Belgians dump -> enrich wikidata Belgians with ISNI

**Phase 2: Enriching**

In [ ]:
# overlap to BnF and KB via ISNI identifiers

In [ ]:
# overlap to BnF and KB via their respective identifiers

## KB NTA

Todo: overlap from KB NTA with Wikidata-belgians and BnF, based on different identifiers.

## BnF authors

Todo: overlap from BnF author persons with Wikidata-belgians and BnF, based on different identifiers.